# Plot channel spectra from the CSV files

Plots channel spectra data from provided CSV files.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from IPython.display import display, Math, Latex
import sys
import gc
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

# Compute channel spectrum for Liulin

In [ ]:
# Import Liulin measurements processed by the timestamp format transformation sed script
surface = 2 # cm2
df = pd.read_csv('liulin-all.dat', sep=',', header=0, parse_dates=[0])
df['flux'] = df.iloc[:,range(3,260)].sum(axis=1)/(surface*df.exposition)
df.set_index('timestamp',inplace=True)

# Detect events, where ch1 > 0, e.g. we assume that there was some interference
# Move these to another dataframe
dfn = df[df['ch1'] > 0]

# Remove that noise from the original dataframe
df.drop(df[df['ch1'] > 0].index,inplace=True)

print(np.shape(df))

In [ ]:
## Compute channel histogram for Liulin data
### Data columns are 5..260
liulin_channelHisto=df.iloc[:,3:260].sum(axis=0).T
print(np.shape(liulin_channelHisto))

# Channel histogram plot

In [ ]:
# Compute channel histogram
def compute_channels( fna ):
    #fna = 'A2-test.csv'
    # Number of lines in the file
    fna_chunksize = 100000
    dfa_chunks = pd.read_csv(fna, sep=',', header=None, comment='*', parse_dates=[0], error_bad_lines=False,chunksize=fna_chunksize)

    skip_chunks = 0
    chunk_number = 0
    lines_parsed = 0
    for chunk in dfa_chunks:
        chunk_number = chunk_number + 1
        if (chunk_number > skip_chunks):
            # Drop where flux above norm
            chunk['flux']=chunk.iloc[:,267:515].sum(axis=1).astype('float64')
            # Tohle je logicke pro Liulin, ale pro nepresamplovana data z Airdosu nezbytně ne.
            # To jsem zkoušel obejít přes ty kvantily.
            chunk.drop(chunk[chunk['flux'] > 0.17*20.48].index, inplace=True)

            # Compute channel histogram in the chunk
            chunk_sum=chunk.iloc[:,3:515].sum(axis=0).T
            if (chunk_number==skip_chunks+1):
                dfa_channelHisto=chunk_sum
            else:
                dfa_channelHisto=dfa_channelHisto+chunk_sum
            lines_parsed+=len(chunk.index)
            print(repr(fna)+' parsed '+repr(lines_parsed)+' lines.')
            del chunk_sum
            del chunk
            gc.collect()
            #print('Histo:')
            #print(dfa_channelHisto)
            #print(chunk_number*fna_chunksize,dfa_channelHisto.sum())
            if ((chunk_number-skip_chunks)*fna_chunksize > 1e8):
                break

            # Beware - resetting column index
            #dfa_channelHisto = dfa_channelHisto.reset_index(drop=True)
        else:
            print('Skipping chunk '+repr(chunk_number))
    del dfa_chunks
    print(repr(fna)+' Done '+repr(lines_parsed)+' lines.')
    return dfa_channelHisto

In [ ]:
# Chunked processor
dfa1_channelHisto=compute_channels('A1-2017-2018-04.csv')
dfa2_channelHisto=compute_channels('A2-2017-2018-04.csv')

In [ ]:
layout = dict(
    title = "Channel histogram - using assumed energy ranges",
    xaxis=dict(
        #range=[dfa1_channelHisto[dfa_channelHisto != 0.0].first_valid_index(),514],
        range=[0.3,4],
        title='Channel energy [MeV]'
    ),
    yaxis=dict(
        #type='log',
        #autorange=True
        range=[0,10000]
    )
)

# Liulin channel width in MeV
liulin_chanwidth=0.0814
liulin_skip=0
data_liulin = go.Scattergl(x=np.arange(0,(256-liulin_skip)*liulin_chanwidth,liulin_chanwidth), y=liulin_channelHisto[liulin_skip:], name='Liulin')

# Bacha, Dasa namita peak
a1min=255
a1emax=6.5
#a1emax=6.0/(2.28/1.47)
print('Estimated energy range: '+repr(a1emax)+' MeV')
data_a1chh = go.Scattergl(x=np.arange(0,a1emax,a1emax/(515-a1min)),y=dfa1_channelHisto[a1min:515],name='A1')

a2min=257
a2emax=a1emax
data_a2chh = go.Scattergl(x=np.arange(0,a2emax,a2emax/(515-a2min)),y=dfa2_channelHisto[a2min:515], name='A2')

fig = dict(data=[data_liulin,data_a1chh,data_a2chh], layout=layout)

plotly.offline.iplot(fig)

In [ ]:
layout = dict(
    title = "Channel histogram - raw channels",
    xaxis=dict(
        #range=[dfa1_channelHisto[dfa_channelHisto != 0.0].first_valid_index(),514],
        #range=[0.3,4],
        title='Channel number'
    ),
    yaxis=dict(
        type='log',
        autorange=True
        #range=[0,10000]
    )
)

# Liulin channel width in MeV
liulin_chanwidth=0.0814
liulin_skip=0
# Rough estimate
data_liulin = go.Scattergl(x=np.arange(256,4*256,4), y=liulin_channelHisto[liulin_skip:], name='Liulin (shifted)')

# Rough channel numbers
data_a1chh = go.Scattergl(y=dfa1_channelHisto[0:515],name='A1')
data_a2chh = go.Scattergl(y=dfa2_channelHisto[0:515], name='A2')

fig = dict(data=[data_liulin,data_a1chh,data_a2chh], layout=layout)

plotly.offline.iplot(fig)

In [ ]:
# If the noise is symmetric, it should be possible to subtract 255..0 from 256..512 
# Subtract noise from first half from second half. 
# Doesn't seem to work right now.
a1min=255
data=dfa1_channelHisto[a1min:]
data.reset_index(inplace=True,drop=True)

noise=np.flip(dfa1_channelHisto[:a1min+1],axis=0)
noise.reset_index(inplace=True,drop=True)
data=data-noise
a1data=data[data>0]

# Subtract noise from first half from second half
a2min=257
data=dfa2_channelHisto[a2min:]
data.reset_index(inplace=True,drop=True)

noise=np.flip(dfa2_channelHisto[:a2min+1],axis=0)
noise.reset_index(inplace=True,drop=True)
data=data-noise
a2data=data[data>0]

layout = dict(
    title = "Channel histogram",
    xaxis=dict(
        #range=[dfa1_channelHisto[dfa_channelHisto != 0.0].first_valid_index(),514],
        range=[0.3,4],
        title='Channel energy [MeV]'
    ),
    yaxis=dict(
        #type='log',
        #autorange=True,
        range=[0,20000]
    )
)

liulin_chanwidth=0.0814
liulin_skip=7
data_liulin = go.Scattergl(x=np.arange(0,(256-liulin_skip)*liulin_chanwidth,liulin_chanwidth), y=liulin_channelHisto[7:], name='Liulin')

a1min=255
a1emax=6.0/(2.28/1.47)
print('Estimated energy range: '+repr(a1emax)+'MeV')
data_a1chh = go.Scattergl(x=np.arange(0,a1emax,a1emax/256),y=a1data, name='A1')

a2min=257
a2emax=a1emax
data_a2chh = go.Scattergl(x=np.arange(0,a2emax,a2emax/256),y=a2data, name='A2')

fig = dict(data=[data_liulin,data_a1chh,data_a2chh], layout=layout)

plotly.offline.iplot(fig)